In [24]:
#!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

In [25]:
import urllib.request

url = "https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py"
urllib.request.urlretrieve(url, "minsearch.py")

('minsearch.py', <http.client.HTTPMessage at 0x1d8c6397e90>)

In [26]:
import minsearch

In [27]:
import json

In [28]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [29]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [30]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [31]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [32]:
q = 'the course has already started, can I still enroll?'

In [33]:
index.fit(documents)

In [34]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [35]:
client = OpenAI()

In [36]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

'This depends on the specific course and its enrollment policies. Here are a few possible scenarios:\n\n1. **Open Enrollment**: Some courses allow enrollment at any time, even after they have started. This is common with online courses or MOOCs (Massive Open Online Courses).\n\n2. **Late Enrollment**: Certain institutions may allow you to enroll late, often within a specified period after the course has begun. There might be penalties or additional requirements, like catching up on missed classes or assignments.\n\n3. **Closed Enrollment**: Some courses have strict enrollment deadlines and do not allow students to join after the course has started. \n\nIt’s best to check with the course provider or institution directly. They can give you the most accurate information regarding their policies and any potential options they might have for late enrollment.'

In [37]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [38]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [39]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [40]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [41]:
rag(query)

'To run Kafka, you can execute the following command in the project directory for Java Kafka:\n\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nIf you are facing issues with running a Python Kafka producer due to module not found errors, you may need to create a virtual environment and execute your Python files within that environment. Here are the steps to set up a virtual environment:\n\n1. Create a virtual environment and install dependencies (run only once):\n   ```bash\n   python -m venv env\n   source env/bin/activate\n   pip install -r ../requirements.txt\n   ```\n\n2. To activate the virtual environment (run each time you work in this environment):\n   ```bash\n   source env/bin/activate\n   ```\n\n3. To deactivate the virtual environment:\n   ```bash\n   deactivate\n   ```\n\nEnsure that Docker images are up and running before running the Python file. Note that the activation path for the virtual environment is 

In [42]:
rag('the course has already started, can I still enroll?')

"Yes, you can still enroll in the course even after it has started. You're also eligible to submit the homework assignments. However, be mindful of the deadlines for turning in the final projects, and try not to leave everything until the last minute."

In [43]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [44]:
from elasticsearch import Elasticsearch

In [49]:
#es_client = Elasticsearch('https://localhost:9200', verify_certs=False)
es_client = Elasticsearch('http://localhost:9200') 

#from elasticsearch import Elasticsearch

#es_client = Elasticsearch(
#    "https://localhost:9200",
#    basic_auth=("elastic", "yourpassword"),
#    verify_certs=False  # Only use verify_certs=False for local/testing/self-signed certs
#)

In [51]:
import requests
print(requests.get("https://localhost:9200", verify=False).text)#print(requests.get("http://localhost:9200/_cat/indices?v").text)


{"error":{"root_cause":[{"type":"security_exception","reason":"missing authentication credentials for REST request [/]","header":{"WWW-Authenticate":["Basic realm=\"security\" charset=\"UTF-8\"","Bearer realm=\"security\"","ApiKey"]}}],"type":"security_exception","reason":"missing authentication credentials for REST request [/]","header":{"WWW-Authenticate":["Basic realm=\"security\" charset=\"UTF-8\"","Bearer realm=\"security\"","ApiKey"]}},"status":401}


c:\Users\mikes\Documents\STUDY\LLM-zoomcamp\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [53]:

es_client = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic", "yourpassword"),
    verify_certs=False  # Only use verify_certs=False for local/testing/self-signed certs
)

c:\Users\mikes\Documents\STUDY\LLM-zoomcamp\venv\Lib\site-packages\elasticsearch\_sync\client\__init__.py:403: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [54]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    }
}

index_name = "test-index"

# Delete the index if it already exists (optional, for a clean start)
if es_client.indices.exists(index=index_name):
    es_client.indices.delete(index=index_name)

# Create the index
#es_client.indices.create(index=index_name, body=index_settings,  request_timeout=60)
es_client.indices.create(
    index=index_name,
    body=index_settings,
    request_timeout=60,
    timeout="30s"
)



c:\Users\mikes\Documents\STUDY\LLM-zoomcamp\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


AuthenticationException: AuthenticationException(401, 'None')

In [68]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client = Elasticsearch("https://localhost:9200", request_timeout=30)


In [ ]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [56]:
from tqdm.auto import tqdm

c:\Users\mikes\Documents\STUDY\LLM-zoomcamp\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [70]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]c:\Users\mikes\Documents\STUDY\LLM-zoomcamp\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
  0%|          | 1/948 [00:00<05:10,  3.05it/s]c:\Users\mikes\Documents\STUDY\LLM-zoomcamp\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\mikes\Documents\STUDY\LLM-zoomcamp\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://u

## to set password:

docker exec -it elasticsearch bin/elasticsearch-reset-password -u elastic

In [71]:
query = 'I just disovered the course. Can I still join it?'

es_client = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic", "Ple_58dNthtgNlQaCCe2"),  # Use the password from your container logs
    verify_certs=False
)

def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

rag(query)

c:\Users\mikes\Documents\STUDY\LLM-zoomcamp\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


'Unfortunately, there is no specific information in the provided context to determine whether you can still join the course. Please check the course enrollment deadlines or contact the course administrator for more information on late admissions.'

In [59]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [62]:
rag(query)

TlsError: TLS error caused by: TlsError(TLS error caused by: SSLError([SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1006)))